In [1]:
import sys
sys.path.insert(0, "/home/alexp/NBA_Models/src/scripts/")

In [2]:
import transform_db
from transform_db import load_clean
from transform_db import transform
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sqlite3

pd.set_option("display.max_columns",None)

In [16]:
path = "/home/alexp/NBA_Models/sqlite/db/nba_data.db"
load = load_clean(path)
agg_boxscores = load.agg_boxscores_raw()
players = load.players()

In [17]:
conn = sqlite3.connect(path)

clean = transform(conn,2013,2023)
game_data = clean.load_team_data()
game_data = clean.clean_team_data(game_data)
game_data = game_data.dropna(subset='PCT_PTS_2PT')
game_data = clean.convert_pcts(game_data)

game_data['TEAM_ID'] = game_data['TEAM_ID'].astype('string')
game_data['GAME_ID'] = game_data['GAME_ID'].astype('string')

transformed_players = players.groupby(["TEAM_ID",'GAME_ID']).mean(numeric_only=True).reset_index()

transformed_players['TEAM_ID'] = transformed_players['TEAM_ID'].astype('string')
transformed_players['GAME_ID'] = transformed_players['GAME_ID'].astype('string')

transformed_players = transformed_players.drop(columns=['WNBA_FANTASY_PTS_RANK','AVAILABLE_FLAG','WNBA_FANTASY_PTS'])


In [18]:
merged_data = pd.merge(left=game_data,right=transformed_players,on=['GAME_ID','TEAM_ID'],suffixes=['_game','_players'])

In [19]:
merged_data = merged_data.drop(columns=['TEAM_ID','GAME_ID','TEAM_NAME','GAME_DATE','MATCHUP','TD3','TD3_RANK'])

In [20]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaled_df = merged_data.loc[:,merged_data.columns[4:]]
scaled_array = scaler.fit_transform(scaled_df)
scaled = pd.DataFrame(scaled_array,columns=scaled_df.columns)

In [21]:
add_columns = merged_data.loc[:,merged_data.columns[:4]]
merged_data[scaled.columns] = scaled.values

# Create Input Array

In [22]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(merged_data['TEAM_ABBREVIATION'])
merged_data['TEAM_ABBREVIATION'] = encoder.transform(merged_data['TEAM_ABBREVIATION'])

encoder.fit(merged_data['SEASON'])
merged_data['SEASON'] = encoder.transform(merged_data['SEASON'])


In [23]:
merged_data

,SEASON,TEAM_ABBREVIATION,HOME_GAME,WL,FG2M,FG2A,FG3M_game,FG3A_game,FTM_game,FTA_game,OREB_game,DREB_game,REB_game,AST_game,STL_game,BLK_game,TOV_game,PF_game,PTS_game,PLUS_MINUS_game,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,E_NET_RATING,NET_RATING,POSS,PIE,PTS_2PT_MR,PTS_FB,PTS_OFF_TOV,PTS_PAINT,AST_2PM,AST_3PM,UAST_2PM,UAST_3PM,MIN,FGM,FGA,FG_PCT,FG3M_players,FG3A_players,FG3_PCT,FTM_players,FTA_players,FT_PCT,OREB_players,DREB_players,REB_players,AST_players,TOV_players,STL_players,BLK_players,BLKA,PF_players,PFD,PTS_players,PLUS_MINUS_players,NBA_FANTASY_PTS,DD2,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,FT_PCT_RANK,OREB_RANK,DREB_RANK,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK
0,0,12,0,0,0.717520,0.584834,-0.687616,-1.105663,-0.755490,0.028718,-0.086941,-0.663888,-0.596407,0.609417,1.156933,-0.342597,0.465315,0.196909,-0.407039,-0.913385,-0.308479,-0.367845,0.138443,0.677734,-0.353448,-0.844217,-0.195618,-0.034419,0.899798,0.823097,1.464703,0.112760,1.038928,-0.460244,-0.224663,-0.400928,0.247028,0.371479,-0.047550,1.170954,-0.576259,-0.904364,0.074424,-0.618033,0.136894,0.362670,0.015112,-0.351929,-0.269961,0.746750,0.741385,1.260987,-0.267458,0.531433,0.345450,0.739201,-0.063576,-1.018323,0.242425,1.468219,0.0,1.063086,1.063086,1.063086,-0.092484,-0.564672,-0.157952,-0.676140,-0.815259,-0.321712,-0.827561,0.681177,0.446753,-0.302252,0.396613,0.125624,0.294807,-0.617594,-0.437307,0.187025,0.557164,0.479244,0.247735,-0.024475,-0.353158,1.159888,0.089244,-0.401328
1,0,13,1,1,-0.227791,0.807075,0.774403,-0.151009,0.079903,0.714149,2.026605,0.070577,1.217892,-0.159385,0.129042,0.459524,1.227177,0.664093,0.573375,0.913372,0.138450,0.677737,-0.308483,-0.367844,0.353457,0.844218,-0.023343,0.034355,0.334836,-0.102217,1.789924,-0.468848,-0.563750,0.658917,0.252394,0.217333,-0.457666,-0.037918,0.073688,0.313076,0.563511,-0.303495,2.336944,-0.081269,0.455428,-0.471460,1.638653,-0.211635,0.613942,-0.349960,1.056588,-0.013536,0.322098,-0.414479,0.326251,-0.062076,0.072279,0.938770,0.069939,0.073847,0.0,-0.983172,-0.983172,-0.983172,0.870505,-0.336209,-0.494568,-0.299893,-1.872066,-1.206355,-2.146114,0.222997,0.011786,0.789382,-0.428507,0.264009,-0.388390,-0.108710,-0.616117,-0.510166,-0.269075,0.006319,0.242983,0.587356,-0.444203,-0.748718,-0.391122,0.387900
2,0,21,0,0,-0.416853,1.918279,-0.443946,-1.344326,-1.925040,-1.753402,0.705638,-1.398352,-0.747599,-1.312587,0.814303,0.459524,1.227177,1.364869,-1.613703,-0.702605,-2.010247,-1.447991,-0.557730,-0.532025,-1.141919,-0.739562,-0.884720,-1.845478,1.182280,-0.410655,0.651651,-1.147390,-1.250612,-0.460244,0.729450,-0.400928,-0.457666,-0.830299,0.073688,-0.960230,-0.532421,-1.321917,0.486101,-1.822125,-1.668135,-1.813289,0.478981,-1.333990,-0.822400,-1.332089,0.606298,0.608183,0.322098,4.678893,0.902235,-1.552823,-1.386900,-0.710063,-1.112827,-1.088129,0.0,1.063086,1.063086,1.063086,0.696707,0.359335,-0.348581,0.935052,-0.786167,0.224251,-0.990957,1.858047,1.991984,1.635722,-0.467056,0.709631,0.431225,0.972886,-0.172153,-0.632276,0.105585,2.732831,0.867438,1.148235,0.728794,0.741297,0.664264,1.045590
3,0,11,1,1,-0.416853,-0.304129,-0.931286,-1.582989,0.748217,1.262494,-0.086941,0.070577,0.008359,-1.312587,-1.241479,5.272251,1.735084,-1.671827,-0.859538,0.702592,-0.557728,-0.532027,-2.010239,-1.447985,1.141927,0.739563,-0.884720,1.845414,0.899798,-0.102217,2.115145,-1.050455,-0.792704,-1.299615,0.252394,0.217333,-0.457666,-1.094426,-1.495278,-0.884967,-0.970794,-1.525602,-0.768497,0.499016,0.927331,0.255064,-0.216823,-0.211635,-0.269961,-1.332089,1.281734,-1.256972,4.743766,0.313146,-1.593697,0.869641,-0.883735,0.723705,-0.308754,-1.088129,0.0,-0.983172,-0.983172,-0.983172,0.614437,0.824677,1.120694,0.422114,-0.348921,0.369282,-0.550137,-0.370124,-0.801712,0.378187,1.142237,0.260149,0.761797,1.057916,-0.784794,1.2911

In [24]:
WL_corr = pd.DataFrame(merged_data.corr()['WL']).reset_index()

In [12]:
import altair as alt

interval = alt.selection_interval()

alt.Chart(WL_corr).mark_bar().encode(
    x='index',
    y='WL',
    tooltip='WL',
    color=alt.Color('WL', scale=alt.Scale(range=['red', 'green']))
)

alt.Chart(...)

In [13]:
# Define two single-value sliders for the lower and upper bounds
min_slider = alt.binding_range(min=-1, max=1, step=0.01, name='Min WL')
max_slider = alt.binding_range(min=-1, max=1, step=0.01, name='Max WL')

# Define the selection for the lower and upper bounds
select_wl_min = alt.selection_single(bind=min_slider, name='Min WL', fields=['WL_min'])
select_wl_max = alt.selection_single(bind=max_slider, name='Max WL', fields=['WL_max'])

# Create the bar chart with the selection filter applied
bar_chart = alt.Chart(WL_corr).mark_bar().encode(
    x='index:N',
    y='WL:Q',
    color=alt.Color('WL', scale=alt.Scale(range=['red', 'green'])
    )
).add_selection(
    select_wl_min,
    select_wl_max
).transform_filter(
    'datum.WL <= Min_WL_WL_min && datum.WL >= Max_WL_WL_max'
)

# Display the chart
bar_chart.display()

/home/alexp/miniconda3/envs/tf/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/home/alexp/miniconda3/envs/tf/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.Chart(...)

In [26]:
WL_pos_corr = WL_corr[(WL_corr['WL'] > .30)]
WL_neg_corr = WL_corr[(WL_corr['WL'] < -.25)]
WL_high_corr = pd.concat([WL_pos_corr,WL_neg_corr])
filter_features = ['TEAM_ABBREVIATION','SEASON','HOME_GAME']+list(WL_high_corr['index'])
data = merged_data[filter_features]
data

,TEAM_ABBREVIATION,SEASON,HOME_GAME,WL,DREB_game,PTS_game,PLUS_MINUS_game,E_OFF_RATING,OFF_RATING,E_NET_RATING,NET_RATING,PIE,PLUS_MINUS_players,NBA_FANTASY_PTS,E_DEF_RATING,DEF_RATING,W_RANK,L_RANK,W_PCT_RANK,FG_PCT_RANK,PLUS_MINUS_RANK
0,12,0,0,0,-0.663888,-0.407039,-0.913385,-0.308479,-0.367845,-0.353448,-0.844217,-0.034419,-1.018323,0.242425,0.138443,0.677734,1.063086,1.063086,1.063086,-0.676140,1.159888
1,13,0,1,1,0.070577,0.573375,0.913372,0.138450,0.677737,0.353457,0.844218,0.034355,0.938770,0.069939,-0.308483,-0.367844,-0.983172,-0.983172,-0.983172,-0.299893,-0.748718
2,21,0,0,0,-1.398352,-1.613703,-0.702605,-2.010247,-1.447991,-1.141919,-0.739562,-1.845478,-0.710063,-1.112827,-0.557730,-0.532025,1.063086,1.063086,1.063086,0.935052,0.741297
3,11,0,1,1,0.070577,-0.859538,0.702592,-0.557728,-0.532027,1.141927,0.739563,1.845414,0.723705,-0.308754,-2.010239,-1.447985,-0.983172,-0.983172,-0.983172,0.422114,-0.771775
4,29,0,0,0,-1.581969,-0.482456,-0.772865,-0.110799,-0.454256,-0.502986,-0.781424,-0.997261,-0.781751,-1.118015,0.516611,0.513552,1.063086,1.063086,1.063086,-0.235247,1.126059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22187,3,10,0,0,-0.663888,-1.312037,-0.913385,-1.030441,-0.998650,-1.148716,-0.976781,-1.524531,-0.925128,-1.265861,0.430664,0.211112,-0.084338,-0.084338,-0.084338,-2.250181,-1.008348
22188,18,10,1,0,-0.847504,-1.010371,-1.124164,-0.832761,-0.713491,-1.223485,-1.186091,-1.249433,-1.044609,-0.993945,0.722885,0.755504,-0.084338,-0.084338,-0.084338,-1.894899,-1.010276
22189,12,10,0,1,1.172274,0.196293,1.124151,0.722895,0.755507,1.223493,1.186091,1.249369,0.977371,-0.825316,-0.832761,-0.713489,-0.983172,-0.983172,-0.983172,-2.506433,-1.799664
22190,17,10,0,1,1.723123,1.025875,1.897010,1.058092,0.885125,1.971181,1.716344,1.971500,1.527639,-0.466789,-1.434392,-1.240598,-0.983172,-0.983172,-0.983172,-2.440701,-2.111602


In [50]:
def create_windows(data,window_size):
  X_list = []
  y_list = []

  for i in range(len(data['TEAM_ABBREVIATION'].unique())):
      team = data.loc[data['TEAM_ABBREVIATION'] == i]
      team_label = team['WL'].copy()
      team = team.drop('WL',axis=1)

      team_np = team.to_numpy()
      team_label = team_label.to_numpy()

      #temp_X = []
      #temp_y = []

      for i in range(len(team_np)-window_size):        
        row = [a for a in team_np[i:i+window_size]]
        X_list.append(row)

        label = team_label[i+window_size]
        y_list.append(label)

  X_list = np.array(X_list)
  y_list = np.array(y_list)

  return X_list,y_list

In [51]:
X_list,y_list = create_windows(data,30)

# Model

In [52]:
from model import ModelConfig

In [53]:
model_config = ModelConfig(X_list,y_list,[.75,.10,.15])

In [54]:
X_train,y_train,X_test,y_test,X_val,y_val = model_config.train_test_val()

In [63]:
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError

model = model_config.create_sequential()
model.add(LSTM(128))
model.add(Dense(8,'relu'))
model.add(Dense(1,'sigmoid'))

model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=.0025), metrics=[RootMeanSquaredError()])

In [64]:
model_config.create_config('model3','/home/alexp/NBA_Models/models/model_configs/')

'Config Created: /home/alexp/NBA_Models/models/model_configs/model3v4.json'

In [65]:
model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=50)

Epoch 1/50
500/500 [==============================] - 3s 4ms/step - loss: 0.2502 - root_mean_squared_error: 0.5002 - val_loss: 0.2500 - val_root_mean_squared_error: 0.5000
Epoch 2/50
500/500 [==============================] - 2s 4ms/step - loss: 0.2501 - root_mean_squared_error: 0.5001 - val_loss: 0.2500 - val_root_mean_squared_error: 0.5000
Epoch 3/50
500/500 [==============================] - 2s 4ms/step - loss: 0.2500 - root_mean_squared_error: 0.5000 - val_loss: 0.2501 - val_root_mean_squared_error: 0.5001
Epoch 4/50
500/500 [==============================] - 2s 4ms/step - loss: 0.2501 - root_mean_squared_error: 0.5001 - val_loss: 0.2500 - val_root_mean_squared_error: 0.5000
Epoch 5/50
500/500 [==============================] - 2s 4ms/step - loss: 0.2501 - root_mean_squared_error: 0.5001 - val_loss: 0.2500 - val_root_mean_squared_error: 0.5000
Epoch 6/50
500/500 [==============================] - 2s 4ms/step - loss: 0.2501 - root_mean_squared_error: 0.5001 - val_loss: 0.2500 - val_

KeyboardInterrupt: 

In [38]:
preds_raw = model.predict(X_test)
actual = y_test
preds = []
for i,pred in enumerate(preds_raw):
    if pred[0] >=.5:
        preds.append(1)
    elif pred[0] <.5:
        preds.append(0)

69/69 [==============================] - 0s 1ms/step


In [39]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

confusion_matrix(actual,preds)

array([[592, 487],
       [469, 642]])

In [40]:
accuracy_score(actual,preds), f1_score(actual,preds)

(0.5634703196347032, 0.5732142857142857)